In [2]:
import pandas as pd
import re
import os
import numpy as np
import datetime

In [5]:
csv_list = []

LUIGI_LOG_PATH = '../data'
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv') and filename.startswith('d'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
appended_df = pd.DataFrame()
#appended_df
csv_list.sort()
csv_list

['daily.2019-July_September.csv']

In [3]:
from difflib import SequenceMatcher 
  
def longestSubstring(str1,str2): 
  
     # initialize SequenceMatcher object with  
     # input string 
     seqMatch = SequenceMatcher(None,str1,str2) 
  
     # find match of longest sub-string 
     # output will be like Match(a=0, b=0, size=5) 
     match = seqMatch.find_longest_match(0, len(str1), 0, len(str2)) 
  
     # print longest substring 
     if (match.size!=0): 
          result = str1[match.a: match.a + match.size]
          print (result)  
          return result
     else: 
          print ('No longest common sub-string found')

In [4]:
PROCESS_NAME = longestSubstring(csv_list[0],csv_list[-1])
OUTPUT_CATT_PATH = LUIGI_LOG_PATH+'/../catts_'+str(PROCESS_NAME)+'.csv'
print(OUTPUT_CATT_PATH)

daily.2019-09-
/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs//../catts_daily.2019-09-.csv


In [5]:
for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('.csv'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        print('Preprocessing... ', filename)
        df = pd.read_csv(log_path, index_col=0)
        df = df[['timestamp','task','state']]
        df['source_file'] = filename

        df['state'][0]='start'
        df['task'][0]='AllTasks'

        df['state'][len(df)-1]='done'
        df['task'][len(df)-1]='AllTasks'
        
        appended_df = appended_df.append(df)
        continue
    else:
        continue

print(len(appended_df))
appended_df.head()

Preprocessing...  daily.2019-09-04_23-23-01.csv
Preprocessing...  daily.2019-09-09_01-34-02.csv
Preprocessing...  daily.2019-09-16_19-44-01.csv
Preprocessing...  daily.2019-09-20_12-49-01.csv
Preprocessing...  daily.2019-09-24_05-05-02.csv
Preprocessing...  daily.2019-09-11_05-40-02.csv
Preprocessing...  daily.2019-09-13_04-11-01.csv
Preprocessing...  daily.2019-09-06_20-34-01.csv
Preprocessing...  daily.2019-09-18_14-08-01.csv
Preprocessing...  daily.2019-09-15_01-41-01.csv
Preprocessing...  daily.2019-09-03_03-44-01.csv
Preprocessing...  daily.2019-09-22_07-40-01.csv
Preprocessing...  daily.2019-09-01_09-29-01.csv
31469


,timestamp,task,state,source_file
0,2019-09-04 23:23:20,AllTasks,start,daily.2019-09-04_23-23-01.csv
1,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",start,daily.2019-09-04_23-23-01.csv
2,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",start,daily.2019-09-04_23-23-01.csv
3,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2...",start,daily.2019-09-04_23-23-01.csv
4,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_...",start,daily.2019-09-04_23-23-01.csv


In [6]:
non_zero = appended_df.dropna()
print(len(non_zero)/2)

pd.set_option('display.max_colwidth', -1)
groupsnontwo = non_zero.groupby(by='task').size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
listnontwo = groupsnontwo[groupsnontwo['counts']!=2]['task'].tolist()[1:]

#non_zero[~non_zero['task'].isin(values)]
#non_zero[non_zero['task'].isin(listnontwo)].sort_values(by=['task', 'state'])

#pot_problems=['CrawlPrivateDataTask(date=2019-09-24_05-05-02, prev_date=2019-09-22_07-40-01, chunk=01)\n', ]

15714.0


In [7]:
def get_task(task_call):
    if isinstance(task_call, float):
        return task_call
    return task_call.split('(')[0]

In [8]:
df = appended_df
df['task_call'] = df['task']
df['task_name'] = np.where(df['task'].notnull(), df['task'].apply(get_task), df['task'])
df = df[['task_name','state','timestamp','task_call', 'source_file']]

df['start_time'] = np.where(df['state']=='start', df['timestamp'], np.nan)
df['end_time'] = np.where(df['state']=='done', df['timestamp'], np.nan)

start_df = df[df['state']=='start']
print('Start entries: ',len(start_df))

end_df = df[df['state']=='done']
print('End entries: ',len(end_df))

df.head()

Start entries:  15729
End entries:  15699


,task_name,state,timestamp,task_call,source_file,start_time,end_time
0,AllTasks,start,2019-09-04 23:23:20,AllTasks,daily.2019-09-04_23-23-01.csv,2019-09-04 23:23:20,NaN
1,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
2,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=regular_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
3,DumpTask,start,2019-09-04 23:35:08,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=reprocess_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN
4,ProxySetupTask,start,2019-09-04 23:35:08,"ProxySetupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep)\n",daily.2019-09-04_23-23-01.csv,2019-09-04 23:35:08,NaN


In [9]:
tt = pd.merge(start_df, end_df, on=['task_call', 'source_file'], how='inner')
tt = tt[tt['start_time_x'].notnull()]
tt = tt[tt['end_time_y'].notnull()]

tt['case'] = tt['source_file']
tt['activity'] = tt['task_name_x']
tt['start_time'] = tt['start_time_x']
tt['end_time'] = tt['end_time_y']
print(len(tt))

15711


## Save catt to csv

In [10]:
catt = tt[['case', 'activity','start_time','end_time']]

#catt.to_csv(OUTPUT_CATT_PATH, index = None, header=True)
print(len(catt))
catt.head()

15711


,case,activity,start_time,end_time
0,daily.2019-09-04_23-23-01.csv,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05
1,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:36:33
2,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-05 00:00:34
3,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:35:58
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,2019-09-04 23:35:08,2019-09-04 23:51:32


In [11]:
pd.options.display.max_colwidth = 100
w_call = tt[['case', 'activity','start_time','end_time', 'task_call']]
grouped = w_call.groupby(['task_call'],as_index=False)
print(len(grouped))
#grouped.head()
#grouped = grouped[grouped['activity']!='CrawlTask']
#grouped.size()

15687


In [12]:
def get_duration(start_time, end_time):
    start = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    end = datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
    duration = abs(end - start)
    return duration
#get_duration(ex['timestamp'][10],ex['timestamp'][1])

In [13]:
w_duration = tt.copy()
w_duration = w_duration[['case', 'activity','start_time','end_time']]
w_duration['duration'] = w_duration.apply(lambda row: str(get_duration(row['start_time'],row['end_time'])), axis=1)
w_duration['t_duration']= w_duration.apply(lambda row: (get_duration(row['start_time'],row['end_time']).total_seconds())/60/60, axis=1)


w_duration = w_duration[['case','activity','start_time','end_time', 'duration', 't_duration']]
print(len(w_duration))
w_duration.head()

15711


,case,activity,start_time,end_time,duration,t_duration
0,daily.2019-09-04_23-23-01.csv,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05,"1 day, 21:09:45",45.162500
1,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:36:33,0:01:25,0.023611
2,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-05 00:00:34,0:25:26,0.423889
3,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:35:58,0:00:50,0.013889
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,2019-09-04 23:35:08,2019-09-04 23:51:32,0:16:24,0.273333


In [14]:
top_long_tasks = w_duration.copy()
max_duration = top_long_tasks.groupby(['activity'], sort=False)['t_duration'].max().reset_index()
#.reset_index(name='max_duration').sort_values(by=['max_duration'], ascending=False)
counts = top_long_tasks.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
act_group = pd.merge(counts, max_duration, on=['activity'], how='outer')
act_group = act_group.sort_values(by=['counts'], ascending = False)
print(len(act_group))
#list(act_group['activity'])

67


### Sanity Check

In [15]:
csv_list = []
print(LUIGI_LOG_PATH)
for filename in os.listdir(LUIGI_LOG_PATH+'../'):
    if filename.endswith('.csv'):
        print(filename)

/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/logs/
catts_daily.2019-09-.csv
catts_daily.2019-09-01_09-29-01.csv
param_catts_daily.2019-09-.csv


In [16]:
csv_list = []
appended_df = pd.DataFrame()
#LUIGI_LOG_PATH='/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/october/'
LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily/'
#LUIGI_LOG_PATH = '/usr/local/trustyou/home/andream/nfs/processmining/minilogs/table-precomp/logs/tch/'

for filename in os.listdir(LUIGI_LOG_PATH):
    if filename.endswith('-.csv') and filename.startswith('catts'):
        log_path = LUIGI_LOG_PATH+'/'+filename
        csv_list.append(filename)
        df = pd.read_csv(log_path, index_col=False)
        appended_df = appended_df.append(df)

#appended_df
csv_list.sort()
print(csv_list)
print(len(appended_df))
appended_df.head()

['catts_daily.2019-09-.csv']
15711


,case,activity,start_time,end_time
0,daily.2019-09-04_23-23-01.csv,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05
1,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:36:33
2,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-05 00:00:34
3,daily.2019-09-04_23-23-01.csv,DumpTask,2019-09-04 23:35:08,2019-09-04 23:35:58
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,2019-09-04 23:35:08,2019-09-04 23:51:32


## Cleaning paramters

In [17]:
task_calls = tt.copy()[['case', 'activity', 'task_call', 'start_time', 'end_time']]
print(len(task_calls))
task_calls.head()

15711


,case,activity,task_call,start_time,end_time
0,daily.2019-09-04_23-23-01.csv,AllTasks,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05
1,daily.2019-09-04_23-23-01.csv,DumpTask,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily...",2019-09-04 23:35:08,2019-09-04 23:36:33
2,daily.2019-09-04_23-23-01.csv,DumpTask,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=regul...",2019-09-04 23:35:08,2019-09-05 00:00:34
3,daily.2019-09-04_23-23-01.csv,DumpTask,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=repro...",2019-09-04 23:35:08,2019-09-04 23:35:58
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,"ProxySetupTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep)\n",2019-09-04 23:35:08,2019-09-04 23:51:32


In [18]:
counts = task_calls.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
print(len(task_calls),' lines in catt')
counts.head()


15711  lines in catt


,activity,counts
19,ExtractTask,6346
14,CrawlTask,6326
17,DumpTask,585
8,ConvertDumpTask,533
24,GroupTask,89


In [19]:
unique_act = task_calls['activity'].unique().tolist()
print(len(unique_act), ' different activities')

unique_trace = task_calls['case'].unique().tolist()
print(len(unique_trace), ' cases\n')

unique_task_call = task_calls['task_call'].unique().tolist()
print(len(unique_task_call), 'unique task_calls')
print(len(task_calls)-len(unique_task_call), 'duplicated task_calls')

pd.set_option('display.max_colwidth', -1)
dfObj = pd.DataFrame(task_calls, columns=['case','task_call','activity','start_time', 'end_time'])
duplicated_task_calls = dfObj[dfObj.duplicated(['case','task_call','end_time'])].sort_values(by=['task_call'])['task_call'].unique().tolist()
print(len(duplicated_task_calls),' unique from duplicated task_calls')

67  different activities
13  cases

15687 unique task_calls
24 duplicated task_calls
9  unique from duplicated task_calls


In [20]:
def extract_parameters(task_call):
    task_split = task_call.split('(', 1)
    parameters = {}
    next_key = None
    key = None
    if len(task_split)>1: 
        reversed_parameters = [''.join(reversed(element)) for element in task_split[1:]][0].split(')', 1)[1:]
        forward_parameters = [''.join(reversed(element)) for element in reversed_parameters][0]
        parameters_call = forward_parameters.split('=')
        #print(task_call)<<<<
        for i, element in enumerate(parameters_call): 
            if next_key is None and key is None: 
                key = element
                continue
            if next_key is not None:
                key = next_key
            if element.startswith('('):
                next_key = re.search('\), (.*)', element)
                value = re.search('(\(.*)\), ', element)
            else: 
                next_key = re.search(', (.*)', element)
                value = re.search('(.*), ', element)
            if next_key is None: 
                #Find out if elem is key or value
                value = element
            else:
                next_key = next_key.group(1)
                if element.startswith('('):
                    value = value.group(0)[:-2]
                else: 
                    value = value.group(1)
            #print('A: ', key, value)               
            parameters[key] = value
    return parameters

#print(extract_parameters('AllTasks'))
#extract_parameters("DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=review, kvs=(('table_name', u'review_2018_3'),), target_filename=review_2018_3, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)")

In [21]:
task_calls['parameters'] = task_calls.apply(lambda row: extract_parameters(row['task_call']), axis=1)
print(len(task_calls))
task_calls.head(2)

15711


,case,activity,task_call,start_time,end_time,parameters
0,daily.2019-09-04_23-23-01.csv,AllTasks,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05,{}
1,daily.2019-09-04_23-23-01.csv,DumpTask,"DumpTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=prep, sql_filename=daily_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n",2019-09-04 23:35:08,2019-09-04 23:36:33,"{'date': '2019-09-04_23-23-01', 'prev_date': '2019-09-03_03-44-01', 'chunk': 'prep', 'sql_filename': 'daily_urls', 'kvs': 'None', 'target_filename': 'None', 'db_host': 'db.trustyou.com', 'db_port': '5432', 'db_user': 'daily', 'db_name': 'ty_analytic'}"


In [22]:
pd.set_option('display.max_columns', None)

expanded_params = pd.concat([task_calls[:], task_calls['parameters'].apply(pd.Series)], axis=1)
len(expanded_params)
#expanded_params.head()

15711

In [23]:
def get_activity_new_name(old_name, column_key, column_value):
    if column_key is None: 
        return old_name
    activity_name = str(old_name)+'('+str(column_key)
    if column_value is np.nan: 
        activity_name = activity_name+'=None'
    else: 
        activity_name = activity_name+'='+str(column_value)
    activity_name = activity_name +')'
    return activity_name

def clean_insignificant_columns(df, drop_candidates):
    BLACKLIST = ['partition']
    WHITELIST = ['parameters', 'task_call']
    columns_to_drop = []
    verified_candidates = []
    number_of_cases = len(df.groupby(['case']))
    flag =0
    
    for group_candidates in drop_candidates:
        for candidate in (set(group_candidates)-set(WHITELIST)): 
            group_counts = df.groupby([candidate]).size().reset_index(name='counts').sort_values(by=['counts'])['counts'].tolist()
            #print(group_counts)
            if ('date' in candidate):
                verified_candidates.append(candidate)
            if len(group_counts)==1 or len(group_counts)==number_of_cases:
                if (all(item == group_counts[0] for item in group_counts) and (group_counts[0]*len(group_counts))== len(df)):
                    #print(group_counts)
                    #or all(item == number_of_cases for item in group_counts)
                    verified_candidates.append(candidate)
            else:
                flag = 1
                break
        if flag:
            first_candidate_counts = df.groupby([group_candidates[0]]).size().reset_index(name='counts')['counts'].tolist()
            if (all(first_candidate_counts==df.groupby([item]).size().reset_index(name='counts')['counts'].tolist() for item in group_candidates)):
                all(verified_candidates.append(item) for item in group_candidates[:-1])
            flag = 0
    columns_to_drop = list(set(verified_candidates)-set(catt.columns)-set(WHITELIST))
    columns_to_drop.extend(list(set(df.columns)&set(BLACKLIST)))

    df = df.drop(columns_to_drop, axis=1)
    #print('Dropped: ',columns_to_drop, ' for activity: ', df['activity'].iloc[0])
    
    #if (set(df.columns)-set(catt.columns)-set({'parameters', 'task_call'})==set()):
    df['activity_parametrized'] = df['activity']
    retained_columns = set(df.columns)-set(catt.columns)-set(WHITELIST)
    if not (retained_columns==set()):
        for column in retained_columns-set({'activity_parametrized'}):
            df['activity_parametrized'] = df.apply(lambda row: get_activity_new_name(row['activity_parametrized'], column, row[column]), axis=1)
    #print(df['activity'].iloc[0], 'was added', retained_columns-set({'activity_parametrized'}))
    return df 

    
activity_selection = expanded_params[expanded_params['activity']=='CrawlTask'].sort_values(by='case')
activity_selection = activity_selection.dropna(axis=1,how='all')
drop_candidates = [['prev_date', 'case', 'date'], ['task_call', 'end_time', 'start_time']]
clean_dump = clean_insignificant_columns(activity_selection, drop_candidates)
#print(clean_dump.head())
counts = clean_dump.groupby(['activity_parametrized']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
counts = counts.sort_values(by=['counts'], ascending = False)

In [24]:
def get_parametrized_activity(df):
    clean_dump = pd.DataFrame()
    return_columns = ['activity_parametrized', 'case', 'activity', 'start_time', 'end_time', 'task_call']
    non_str_columns = ['parameters']
    df = df.replace('None', np.nan)
    activities = df.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)['activity'].tolist()
    for activity in activities:
        activity_selection = df[df['activity']==activity].sort_values(by='case')
        activity_selection = activity_selection.dropna(axis=1,how='all')
        column_selection = set(activity_selection.columns) - set(non_str_columns)
        
        groups_description = pd.DataFrame(columns=['column','different_groups_per_column'])
        for column in column_selection: 
            #print(column,len(clean_activity_selection.groupby([column]).size().reset_index(name='counts').sort_values(by=['counts'])))
            groups_description = groups_description.append([{'column': column, 'different_groups_per_column': len(activity_selection.groupby([column]).size().reset_index(name='counts').sort_values(by=['counts']))}])
        groups_description = groups_description.reset_index()[['column','different_groups_per_column']].sort_values(by=['different_groups_per_column'])
        
        similar_group_descr = groups_description.groupby(by=['different_groups_per_column']).size().reset_index(name='similar_group')
        similar_group_counts = similar_group_descr[similar_group_descr['similar_group']>1]['different_groups_per_column'].tolist()
        #print('Similar number of groups with multiple appereances in Groupbys: ', similar_group_counts)

        drop_candidates = []
        for group in similar_group_counts: 
            parameters = groups_description[groups_description['different_groups_per_column']==group]['column'].tolist()
            drop_candidates.append(parameters)
        #print('Groups with similar number of groups: ', drop_candidates)
        clean_dump = clean_dump.append(clean_insignificant_columns(activity_selection, drop_candidates)[return_columns])
    df = pd.merge(df, clean_dump, on=return_columns[1:], how='inner')[return_columns]
    return df

        #print(activity_selection.columns)

subset = expanded_params[expanded_params['activity'].str.startswith('CrawlTask') & expanded_params['case'].isin(['daily.2019-09-04_23-23-01.csv','daily.2019-09-09_01-34-02.csv'])]
get_parametrized_activity(subset).head()

,activity_parametrized,case,activity,start_time,end_time,task_call
0,CrawlTask(chunk=01)(crawler=ty-superman),daily.2019-09-04_23-23-01.csv,CrawlTask,2019-09-05 01:13:08,2019-09-05 01:13:48,"CrawlTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01, partition=00000, crawler=ty-superman)\n"
1,CrawlTask(chunk=03)(crawler=ty-superman),daily.2019-09-04_23-23-01.csv,CrawlTask,2019-09-05 01:13:08,2019-09-05 01:13:50,"CrawlTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=03, partition=00000, crawler=ty-superman)\n"
2,CrawlTask(chunk=01)(crawler=creepy-crawly),daily.2019-09-04_23-23-01.csv,CrawlTask,2019-09-05 01:17:31,2019-09-05 04:31:10,"CrawlTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01, partition=00239, crawler=creepy-crawly)\n"
3,CrawlTask(chunk=01)(crawler=creepy-crawly),daily.2019-09-04_23-23-01.csv,CrawlTask,2019-09-05 01:17:31,2019-09-05 04:03:50,"CrawlTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01, partition=00238, crawler=creepy-crawly)\n"
4,CrawlTask(chunk=01)(crawler=creepy-crawly),daily.2019-09-04_23-23-01.csv,CrawlTask,2019-09-05 01:17:31,2019-09-05 03:51:26,"CrawlTask(date=2019-09-04_23-23-01, prev_date=2019-09-03_03-44-01, chunk=01, partition=00237, crawler=creepy-crawly)\n"


In [25]:
params_call = get_parametrized_activity(expanded_params)
params_call['activity'] = params_call['activity_parametrized']
print(len(params_call), 'unique activity calls')
params_call.head()

#expanded_params.groupby(['activity']).reset_index(name='counts').sort_values(by=['counts'], ascending=False)
counts = params_call.groupby(['activity_parametrized']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
counts = counts.sort_values(by=['counts'], ascending = False)
print(len(counts), 'unique parametrized activities')
counts.head()

#params_call[params_call['activity'].str.startswith('CrawlTask')].head()

15711 unique activity calls
254 unique parametrized activities


,activity_parametrized,counts
73,CrawlTask(chunk=01)(crawler=creepy-crawly),3120
134,ExtractTask(chunk=01)(crawler=creepy-crawly),3120
138,ExtractTask(chunk=03)(crawler=creepy-crawly),1745
77,CrawlTask(chunk=03)(crawler=creepy-crawly),1727
136,ExtractTask(chunk=02)(crawler=creepy-crawly),1443


In [26]:
short_test = params_call[params_call['case']=='daily.2019-09-01_09-29-01.csv'].drop_duplicates()
print(len(short_test))
short_test.head()

1248


,activity_parametrized,case,activity,start_time,end_time,task_call
14463,AllTasks,daily.2019-09-01_09-29-01.csv,AllTasks,2019-09-01 09:29:23,2019-09-03 03:43:13,AllTasks
14464,DumpTask(sql_filename=daily_urls)(kvs=None)(chunk=prep),daily.2019-09-01_09-29-01.csv,DumpTask(sql_filename=daily_urls)(kvs=None)(chunk=prep),2019-09-01 09:42:15,2019-09-01 09:43:44,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=daily_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n"
14465,DumpTask(sql_filename=regular_urls)(kvs=None)(chunk=prep),daily.2019-09-01_09-29-01.csv,DumpTask(sql_filename=regular_urls)(kvs=None)(chunk=prep),2019-09-01 09:42:15,2019-09-01 10:09:02,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=regular_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n"
14466,DumpTask(sql_filename=other_urls)(kvs=None)(chunk=prep),daily.2019-09-01_09-29-01.csv,DumpTask(sql_filename=other_urls)(kvs=None)(chunk=prep),2019-09-01 09:42:15,2019-09-01 09:42:54,"DumpTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep, sql_filename=other_urls, kvs=None, target_filename=None, db_host=db.trustyou.com, db_port=5432, db_user=daily, db_name=ty_analytic)\n"
14467,ProxySetupTask,daily.2019-09-01_09-29-01.csv,ProxySetupTask,2019-09-01 09:42:15,2019-09-01 09:54:13,"ProxySetupTask(date=2019-09-01_09-29-01, prev_date=2019-08-30_13-45-01, chunk=prep)\n"


### Save to csv

In [27]:
params_catt = params_call[['case', 'activity','start_time','end_time']]

PROCESS_NAME = longestSubstring(csv_list[0],csv_list[-1])
OUTPUT_PARAM_CATT_PATH = LUIGI_LOG_PATH+'/./param_'+str(PROCESS_NAME[:-4])+'.csv'
print(OUTPUT_PARAM_CATT_PATH)

#params_catt.to_csv(OUTPUT_PARAM_CATT_PATH, index = None, header=True)
print(len(params_catt))
params_catt.head()

catts_daily.2019-09-.csv
/usr/local/trustyou/home/andream/nfs/processmining/minilogs/daily//./param_catts_daily.2019-09-.csv
15711


,case,activity,start_time,end_time
0,daily.2019-09-04_23-23-01.csv,AllTasks,2019-09-04 23:23:20,2019-09-06 20:33:05
1,daily.2019-09-04_23-23-01.csv,DumpTask(sql_filename=daily_urls)(kvs=None)(chunk=prep),2019-09-04 23:35:08,2019-09-04 23:36:33
2,daily.2019-09-04_23-23-01.csv,DumpTask(sql_filename=regular_urls)(kvs=None)(chunk=prep),2019-09-04 23:35:08,2019-09-05 00:00:34
3,daily.2019-09-04_23-23-01.csv,DumpTask(sql_filename=reprocess_urls)(kvs=None)(chunk=prep),2019-09-04 23:35:08,2019-09-04 23:35:58
4,daily.2019-09-04_23-23-01.csv,ProxySetupTask,2019-09-04 23:35:08,2019-09-04 23:51:32


### Realization from looking into logs: 

For every distinct activity logged, one distinct start_time and one distinct end_time are expected. However: 

When a task is restarted module saves as many points for that task as times it was started, all share same end_time. (E.g. GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo))

```bash
andream@dev-box in ~/git_tree/processmining on master [!?]
$ cat ~/nfs/processmining/minilogs/daily/logs/daily.2019-09-22_07-40-01.log | grep 'GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)' | grep '\[pid'
2019-09-23 04:41:43,299 INFO     worker.py:58   - [pid 14957] Worker Worker(salt=352744368, host=vmw.trustyou.com, username=daily-user, pid=22331) running   GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)
2019-09-24 02:10:28,725 ERROR    worker.py:88   - [pid 14957] Worker Worker(salt=352744368, host=vmw.trustyou.com, username=daily-user, pid=22331) failed    GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)
2019-09-24 02:26:21,494 INFO     worker.py:58   - [pid 30935] Worker Worker(salt=352744368, host=vmw.trustyou.com, username=daily-user, pid=22331) running   GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)
2019-09-24 03:09:23,583 INFO     worker.py:80   - [pid 30935] Worker Worker(salt=352744368, host=vmw.trustyou.com, username=daily-user, pid=22331) done      GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)
````

Other tasks (E.g. QCUpdateCrawlDateTask(date=2019-09-24_05-05-02, prev_date=2019-09-22_07-40-01, chunk=01)) show start_time row in log only. No 'done', no 'failed', etc.

```bash
andream@dev-box in ~/git_tree/processmining on master [!?]
$ cat ~/nfs/processmining/minilogs/daily/logs/daily.2019-09-24_05-05-02.log | grep 'QCUpdateCrawlDateTask(date=2019-09-24_05-05-02, prev_date=2019-09-22_07-40-01, chunk=01)' | grep '\[pid'
2019-09-24 16:43:14,079 INFO     worker.py:58   - [pid 13179] Worker Worker(salt=187047613, host=vmw.trustyou.com, username=daily-user, pid=19144) running   QCUpdateCrawlDateTask(date=2019-09-24_05-05-02, prev_date=2019-09-22_07-40-01, chunk=01)
````

In [28]:
non_zero[non_zero['task']=='QCUpdateCrawlDateTask(date=2019-09-24_05-05-02, prev_date=2019-09-22_07-40-01, chunk=01)\n']
non_zero[non_zero['task']=='GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)\n']

,timestamp,task,state,source_file
1958,2019-09-23 04:41:43,"GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)\n",start,daily.2019-09-22_07-40-01.csv
2445,2019-09-24 02:26:21,"GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)\n",start,daily.2019-09-22_07-40-01.csv
2475,2019-09-24 03:09:23,"GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)\n",done,daily.2019-09-22_07-40-01.csv


In [29]:
tt[tt['activity']=='QCUpdateCrawlDateTask(date=2019-09-24_05-05-02, prev_date=2019-09-22_07-40-01, chunk=01)\n']
tt[tt['activity']=='GroupTask(date=2019-09-22_07-40-01, prev_date=2019-09-20_12-49-01, chunk=01, datacenter=eu, inc=False, deploy=False, datastore=mongo)\n']

,task_name_x,state_x,timestamp_x,task_call,source_file,start_time_x,end_time_x,task_name_y,state_y,timestamp_y,start_time_y,end_time_y,case,activity,start_time,end_time


### Other started experiments: 

In [30]:
subset = params_catt[params_catt['activity'].str.startswith('CrawlTask') & params_catt['case'].isin(['daily.2019-09-04_23-23-01.csv','daily.2019-09-09_01-34-02.csv'])]
print(len(subset))
subset.head()

986


,case,activity,start_time,end_time
34,daily.2019-09-04_23-23-01.csv,CrawlTask(chunk=01)(crawler=ty-superman),2019-09-05 01:13:08,2019-09-05 01:13:48
35,daily.2019-09-04_23-23-01.csv,CrawlTask(chunk=03)(crawler=ty-superman),2019-09-05 01:13:08,2019-09-05 01:13:50
44,daily.2019-09-04_23-23-01.csv,CrawlTask(chunk=01)(crawler=creepy-crawly),2019-09-05 01:17:31,2019-09-05 04:31:10
45,daily.2019-09-04_23-23-01.csv,CrawlTask(chunk=01)(crawler=creepy-crawly),2019-09-05 01:17:31,2019-09-05 04:03:50
46,daily.2019-09-04_23-23-01.csv,CrawlTask(chunk=01)(crawler=creepy-crawly),2019-09-05 01:17:31,2019-09-05 03:51:26


In [31]:

counts = subset.groupby(['activity']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=True)
print(len(subset),' lines in catt')
print(len(counts), 'different activities')
counts

#counts[counts['activity'].str.startswith('GroupTask(')].head()
counts[~counts['activity'].str.endswith(')')]

986  lines in catt
6 different activities


,activity,counts
